# Basic modelling in AuTuMN

This notebook provides a brief overview the interface to running and interacting with models in AuTuMN  

In [ ]:
# Start with imports
# These should always live at the top of a notebook

# Import commonly used external libraries - you want always need these, but most of the time you will
import numpy as np
import pandas as pd

# Import our project interface - this is the main method of accessing our models
from autumn.tools.project import get_project

## Projects

Projects encapsulate everything with we need to interact with a given model  
This includes parameters, calibration specifications, and tools for building and running the model itself

In [ ]:
# To open a project, we need to know the model name (ie the type of model we are using)
# and the model region, which will have its own input data, calibration targets etc

model_name = 'covid_19'
model_region = 'malaysia'

p = get_project(model_name, model_region)

In [ ]:
# Run the model with unmodified baseline parameters
# This command returns a summer CompartmentalModel object that contains the completed run data
# We will detail how to modify and interact with parameters in the next notebook

params = p.param_set.baseline

m = p.run_baseline_model(params)

## Model outputs

There are 2 kinds of output data in AuTuMN/summer models

1. Outputs (model.outputs, model.get_outputs_df)  
The 'raw' model outputs - ie the value of every compartment at every timestep
2. Derived outputs (model.derived_outputs, model.get_derived_outputs_df)  
Additional outputs that are computed after the model is run, which may be combinations of the other outputs, or have transformations applied etc

In [ ]:
# The raw model outputs are a numpy array of shape (times, compartments)
# They can be accessed via model.outputs

# For interactive use, we provide a much more friendly interface in form of pandas DataFrames
# You can interact with these as you would any other tabular data
# Please refer to the pandas documentation for full details

odf = m.get_outputs_df()

odf

In [ ]:
# Here we plot a single compartment

odf['late_exposedXagegroup_20Xclinical_hospital_non_icuXtracing_untracedXstrain_deltaXvaccination_unvaccinated'].plot()

In [ ]:
# In the following example, we compare the clinical ICU impacts of all vaccinated and unvaccinated compartments

In [ ]:
# Select a subset of columns (compartments) by filtering on their names
columns = odf.columns
columns = [c for c in columns if "clinical_icu" in c]
c_vacc = [c for c in columns if "vaccination_vaccinated" in c]
c_unvacc = [c for c in columns if "vaccination_unvaccinated" in c]

In [ ]:
# Construct a new DataFrame using our column selections, and plot it

analysis_df = pd.DataFrame()
analysis_df['vacc'] = odf[c_vacc].sum(axis=1)
analysis_df['unvacc'] = odf[c_unvacc].sum(axis=1)
analysis_df.plot(title='ICU impact by vaccination status')

In [ ]:
# Derived outputs function in exactly the same way

do_df = m.get_derived_outputs_df()

do_df[['accum_deaths','incidence']].plot()